In [2]:
from gurobipy import *
import math


In [12]:

def distance(a,b):
  dx = a[0] - b[0]
  dy = a[1] - b[1]
  return math.sqrt(dx*dx + dy*dy)

# Problem data
clients = [[100, 80],[15, 119],[76,145],[13,33],
           [80,132],[80,112],[0,0],[40,6],
           [120,139],[86,40],[20,60],[66,73]]
facilities = [[100,80],[80,112],[86,40],[20,60],[66,73]]
charge = [10000,10000,10000,10000,10000]
demand = [425,12,43,125,110,86,129,28,66,320,220,182]

numFacilities = len(facilities)
numClients = len(clients)

m = Model()

# Add variables
x = {}
y = {}
d = {} # Distance matrix (not a variable)
alpha = 1

for j in range(numFacilities):
  x[j] = m.addVar(vtype=GRB.BINARY, name="x%d" % j)

for i in range(numClients):
  for j in range(numFacilities):
    y[(i,j)] = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="t%d,%d" % (i,j))
    d[(i,j)] = distance(clients[i], facilities[j])

m.update()

# Add constraints
for i in range(numClients):
  for j in range(numFacilities):
    m.addConstr(y[(i,j)] <= x[j])



for i in range(numClients):
  m.addConstr(quicksum(y[(i,j)] for j in range(numFacilities)) == 1)

m.setObjective( quicksum(charge[j]*x[j] + quicksum(alpha*d[(i,j)]*y[(i,j)]
                for i in range(numClients)) for j in range(numFacilities)) )

m.optimize()

print(d)

Optimize a model with 72 rows, 65 columns and 180 nonzeros
Variable types: 60 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 72 rows, 65 columns, 180 nonzeros
Variable types: 60 continuous, 5 integer (5 binary)

Root relaxation: objective 1.068651e+04, 49 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    10686.505519 10686.5055  0.00%     -    0s

Explored 0 nodes (49 simplex iterations) in 0.10 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 10686.5 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.068650551911e+04, best bound 1.068650551911e+04, gap 0.0000%
{(0, 0): 0.0, (0, 1): 37.73592452822641, (0, 2): 42.379240